# Energy Efficiency Dataset – EDA and Initial Cleaning

## 1. Introduction
This notebook documents the **Exploratory Data Analysis (EDA)** and the **initial cleaning phase** for the Energy Efficiency dataset (UCI).  
The dataset describes the geometrical and physical characteristics of buildings (`X1–X8`) and their energy loads (`Y1`, `Y2`).  

The purpose of this first notebook is to:
- Explore the raw noisy dataset.
- Detect and understand missing, invalid, and inconsistent values.
- Decide how to handle missing data, generating **two candidate cleaning approaches**:
  1. **Drop missing values** (strict version).
  2. **Fill missing values** (preserve data version).  

Subsequent notebooks will handle deeper cleaning, outlier detection, and final preprocessing steps for each branch.

---

## 2. Data Loading and Overview

The raw dataset (`data/raw/energy_efficiency_modified.csv`) was loaded using pandas.
The original dataset is (`data/raw/energy_efficiency_original.csv`) and the documentation is available at the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/242/energy+efficiency).

---

## 3. Data Cleaning (Initial Phase)
### **Type correction**
- Converted all numeric features using `pd.to_numeric(errors='coerce')`.
- Invalid textual entries were replaced with `NaN`.

---

## 4. Missing-Value Handling Strategies
To preserve reproducibility and facilitate comparison, two alternative cleaning strategies were implemented:

1. **Drop approach (`df_drop`)**  
2. **Fill approach (`df_fill`)**  

---

## 5. Comparison Overview
Both datasets were stored for versioning under:
- `data/interim/energy_drop.csv`
- `data/interim/energy_fill.csv`

Each will be analyzed in separate notebooks focusing on:
- Outlier detection and treatment.  
- Feature distribution analysis.  
- Correlation and statistical relationships.  

---


### Imports for the code
This section includes all necessary imports for data manipulation, visualization, and analysis.

In [1]:
import pandas as pd
import seaborn as sns
from IPython.display import display

sns.set_theme(style='whitegrid')

### Load Dataset
This section loads the dataset from the specified path and displays the first few rows for an initial overview.

In [2]:
DATA_PATH = '../data/raw/energy_efficiency_modified.csv'
df = pd.read_csv(DATA_PATH)
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,mixed_type_col
0,0.98,514.5,294.0,110.25,7.0,2.0,0.0,0.0,15.55,21.33,637
1,0.98,514.5,294.0,110.25,7.0,3.0,0.0,0.0,15.55,21.33,618
2,0.98,514.5,294.0,110.25,7.0,4.0,0.0,0.0,15.55,21.33,412
3,0.98,514.5,294.0,110.25,7.0,5.0,0.0,0.0,15.55,21.33,243
4,0.9,563.5,318.5,122.5,7.0,2.0,0.0,0.0,20.84,28.28,979


### Data understanding and initial overview
This section provides a comprehensive overview of the dataset, including its shape, data types, missing values, and basic statistics.

In [3]:
# * Code to display detailed DataFrame info and statistics
pd.set_option('display.max_columns', None)
sns.set(style="whitegrid", palette="pastel")

# Basic info
print("Dataset Shape:", df.shape)
print("-------"*10)
print("Dataset Info:")
df.info()
print("-------"*10)
# Basic statistics (include categorical)
print("Dataset Description:")
display(df.describe(include='all'))
print("-------"*10)
# Count missing values
print("Missing Values per Column:")
display(df.isnull().sum())
print("-------"*10)
# Check data types
print("Data Types:")
display(df.dtypes)


Dataset Shape: (783, 11)
----------------------------------------------------------------------
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   X1              776 non-null    object 
 1   X2              774 non-null    object 
 2   X3              776 non-null    object 
 3   X4              776 non-null    object 
 4   X5              767 non-null    object 
 5   X6              772 non-null    object 
 6   X7              771 non-null    object 
 7   X8              772 non-null    float64
 8   Y1              777 non-null    object 
 9   Y2              777 non-null    object 
 10  mixed_type_col  696 non-null    object 
dtypes: float64(1), object(10)
memory usage: 67.4+ KB
----------------------------------------------------------------------
Dataset Description:


,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2,mixed_type_col
count,776,774,776,776,767,772,771,772.000000,777,777,696
unique,31,32,25,19,16,20,16,NaN,596,645,423
top,0.79,637.0,294.0,220.5,7.0,4.0,0.4,NaN,15.16,29.79,bad
freq,64,65,179,355,362,185,231,NaN,6,4,76
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.200777,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.220112,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN


----------------------------------------------------------------------
Missing Values per Column:


X1                 7
X2                 9
X3                 7
X4                 7
X5                16
X6                11
X7                12
X8                11
Y1                 6
Y2                 6
mixed_type_col    87
dtype: int64

----------------------------------------------------------------------
Data Types:


X1                 object
X2                 object
X3                 object
X4                 object
X5                 object
X6                 object
X7                 object
X8                float64
Y1                 object
Y2                 object
mixed_type_col     object
dtype: object

### Normalizing Column Types
This section focuses on identifying and correcting inconsistent data types across columns, ensuring that each column has the appropriate type for analysis.
First, we create a quick profile of each column to identify potential issues.
Then, we specifically check columns that should be numeric but are stored as objects due to invalid entries. We convert these columns to numeric types, coercing errors to NaN.
Finally, we recheck the data types and count any NaNs created during the conversion.

In [4]:
# * Quick profile per column
summary = []
for col in df.columns:
    unique_count = df[col].nunique(dropna=True)
    na_count = df[col].isna().sum()
    example_values = df[col].dropna().unique()[:10]  # first 10 unique examples
    summary.append([col, df[col].dtype, unique_count, na_count, example_values])

profile_df = pd.DataFrame(summary, columns=["Column", "Dtype", "Unique Values", "Null Count", "Examples"])
profile_df

,Column,Dtype,Unique Values,Null Count,Examples
0,X1,object,31,7,"[0.98, 0.9, 0.86, 0.82, 0.79, 0.76, 0.74, 0.71..."
1,X2,object,32,9,"[514.5, 563.5, 588.0, 612.5, 637.0, 661.5, 686..."
2,X3,object,25,7,"[294.0, 318.5, 318.5 , 343.0 , 343.0, 416.5,..."
3,X4,object,19,7,"[110.25, 110.25 , 122.5, 147.0, 122.5 , 220...."
4,X5,object,16,16,"[7.0, 7.0 , 3.5 , 3.5, 269.5, 126.0, 806.0, ..."
5,X6,object,20,11,"[2.0, 3.0, 4.0, 5.0, 4.0 , 5.0 , 3.0 , 244...."
6,X7,object,16,12,"[0.0, 516.0, 0.1, 0.1 , 1.0, 0.25, 0.25 , 10..."
7,X8,float64,10,11,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 971.0, 316.0, 4..."
8,Y1,object,596,6,"[15.55, 20.84, 21.46, 20.71, 19.68, 19.5, 19.9..."
9,Y2,object,645,6,"[21.33, 28.28, 25.38, 25.16, 29.6, 27.3, 21.97..."


In [5]:
# * Code to check columns that should be numeric but are object type and give stats about it

import re

#Dropping the extra column
df = df.drop(columns=['mixed_type_col'], errors='ignore')
print("Remaining columns:", df.columns.tolist())
print("Shape after drop:", df.shape)

def count_invalid_entries(series):
    """Return count of cells that are not valid numeric strings."""
    invalid = 0
    for v in series.dropna():
        # Accept numbers (integers or floats)
        if not re.match(r'^-?\d+(\.\d+)?$', str(v).strip()):
            invalid += 1
    return invalid

invalid_summary = []
for col in ['X1','X2','X3','X4','X5','X6','X7','X8','Y1','Y2']:
    if df[col].dtype == 'object':
        invalid_count = count_invalid_entries(df[col])
        invalid_summary.append((col, invalid_count, len(df[col]), round(invalid_count/len(df[col])*100,2)))

invalid_df = pd.DataFrame(invalid_summary, columns=['Column','Invalid_Count','Total_Rows','Invalid_%'])
invalid_df

Remaining columns: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'Y1', 'Y2']
Shape after drop: (783, 10)


,Column,Invalid_Count,Total_Rows,Invalid_%
0,X1,2,783,0.26
1,X2,1,783,0.13
2,X3,6,783,0.77
3,X4,3,783,0.38
4,X5,3,783,0.38
5,X6,2,783,0.26
6,X7,2,783,0.26
7,Y1,1,783,0.13
8,Y2,3,783,0.38


In [6]:
# * Code to coerce conversion of numeric columns, forcing invalid entries to NaN

# ? Columns we know should be numeric
num_cols = ['X1','X2','X3','X4','X5','X6','X7','X8','Y1','Y2']

# ? Coerce conversion (invalid entries -> NaN)
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# ? X6 and X8 stay categorical
df['X6'] = df['X6'].astype('category')
df['X8'] = df['X8'].astype('category')

# ? Recheck
df[num_cols].info()

# ? Count NaNs created
df[num_cols].isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 783 entries, 0 to 782
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype   
---  ------  --------------  -----   
 0   X1      774 non-null    float64 
 1   X2      773 non-null    float64 
 2   X3      770 non-null    float64 
 3   X4      773 non-null    float64 
 4   X5      764 non-null    float64 
 5   X6      770 non-null    category
 6   X7      769 non-null    float64 
 7   X8      772 non-null    category
 8   Y1      776 non-null    float64 
 9   Y2      774 non-null    float64 
dtypes: category(2), float64(8)
memory usage: 51.6 KB


X1     9
X2    10
X3    13
X4    10
X5    19
X6    13
X7    14
X8    11
Y1     7
Y2     9
dtype: int64

### Missing Values Overview
This section provides a detailed overview of missing values in the dataset, including counts and percentages per column. It also sets up two different strategies for handling missing data: dropping rows with missing values and filling (imputing) missing values.

In [7]:
# * Code to calculate number and percentage of NaNs per column
nan_table = (
    df.isna()
      .sum()
      .to_frame(name='NaN Count')
      .assign(
          Total_Rows=len(df),
          NaN_Percent=lambda x: (x['NaN Count'] / x['Total_Rows'] * 100).round(2)
      )
      .sort_values(by='NaN_Percent', ascending=False)
)

nan_table

,NaN Count,Total_Rows,NaN_Percent
X5,19,783,2.43
X7,14,783,1.79
X6,13,783,1.66
X3,13,783,1.66
X8,11,783,1.40
X2,10,783,1.28
X4,10,783,1.28
X1,9,783,1.15
Y2,9,783,1.15
Y1,7,783,0.89


In [8]:
# * Code to create two versions of the dataset: one dropping NaNs, one filling NaNs
# ? --- Base copy (after coercion, before null handling) ---
df_base = df.copy()

# ? --- Version A: Drop rows with NaN ---
df_drop = df_base.dropna().reset_index(drop=True)

# ? --- Version B: Fill (impute) missing values ---
df_fill = df_base.copy()

# ? Fill continuous (median)
num_cols = ['X1','X2','X3','X4','X5','X7','Y1','Y2']
for c in num_cols:
    df_fill[c] = df_fill[c].fillna(df_fill[c].median())

# ? Fill categorical (mode)
for c in ['X6','X8']:
    df_fill[c] = df_fill[c].fillna(df_fill[c].mode()[0])

# ? --- Quick checks ---
print("Shape original:", df_base.shape)
print("Shape drop version:", df_drop.shape)
print("Shape fill version:", df_fill.shape)

print("\nRemaining NaNs in df_drop:", df_drop.isna().sum().sum())
print("Remaining NaNs in df_fill:", df_fill.isna().sum().sum())

Shape original: (783, 10)
Shape drop version: (680, 10)
Shape fill version: (783, 10)

Remaining NaNs in df_drop: 0
Remaining NaNs in df_fill: 0


### Saving the datasets to files and then loading them back separately
This section saves the two cleaned versions of the dataset (one with dropped NaNs and one with filled NaNs) to CSV and Parquet formats for future use in the interim data directory.

In [9]:
# * Code to save each version

# ? Cleaned version — dropped NaNs
df_drop.to_csv("../data/interim/energy_efficiency_interim_clean_drop.csv", index=False)

# ? Cleaned version — filled NaNs
df_fill.to_csv("../data/interim/energy_efficiency_interim_clean_fill.csv", index=False)

print("All datasets saved successfully.")

All datasets saved successfully.
